## Load cleaned datasets

In [2]:
import pandas as pd

all_datasets_df = pd.read_csv('data/all_datasets_merged.csv')
print('Loaded all datasets merged:', len(all_datasets_df))

primock_datasets = pd.read_csv('data/primock_uk_datasets_processed.csv')
print('Loaded primock datasets:', len(primock_datasets))

Loaded all datasets merged: 90
Loaded primock datasets: 45


## Phi-4 Model

In [2]:
import os
import requests
import torch
from PIL import Image
import soundfile
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig

# import datasets
from datasets import load_dataset
import torchcodec
import os
from tqdm import tqdm

/home/kelechi/miniconda3/envs/phi4_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load model and processor
model_path = "kumapo/Phi-4-multimodal-instruct"
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True,
    _attn_implementation='eager',
).cuda()
generation_config = GenerationConfig.from_pretrained(model_path)

/home/kelechi/miniconda3/envs/phi4_env/lib/python3.9/site-packages/transformers/models/auto/image_processing_auto.py:646: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
`torch_dtype` is deprecated! Use `dtype` instead!
Phi4MMModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other relat

In [ ]:
user_prompt = '<|user|>'
assistant_prompt = '<|assistant|>'
prompt_suffix = '<|end|>'
speech_prompt = "Based on the attached audio, generate a comprehensive text transcription of the spoken content."

def transcribe_file_chunked(audio_path, chunk_seconds=100, max_new_tokens=500):
    if not os.path.exists(audio_path):
        return f'FILE_NOT_FOUND: {audio_path}'
    try:
        data, sr = soundfile.read(audio_path)  # (numpy array, sample_rate)
        # ensure mono (if stereo, average channels)
        if data.ndim > 1:
            data = data.mean(axis=1)
        total_samples = data.shape[0]
        chunk_samples = int(chunk_seconds * sr)
        if chunk_samples <= 0:
            return "ERROR: invalid chunk_seconds"
        segments = []
        for start in range(0, total_samples, chunk_samples):
            seg = data[start : start + chunk_samples]
            if seg.size == 0:
                continue
            segments.append((seg, sr))
        device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
        texts = []
        for i, seg in enumerate(segments):
            prompt = f'{user_prompt}<|audio_1|>{speech_prompt}{prompt_suffix}{assistant_prompt}'
            inputs = processor(text=prompt, audios=[seg], return_tensors='pt').to(device)
            generate_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                generation_config=generation_config,
                use_cache=False
            )
            # slice off prompt tokens
            generate_ids = generate_ids[:, inputs['input_ids'].shape[1] : ]
            resp = processor.batch_decode(
                generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
            )[0].strip()
            texts.append(resp)
            # cleanup to reduce peak memory
            try:
                del inputs, generate_ids
            except Exception:
                pass
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        # join segment transcriptions (you can change separator)
        return " ".join(t for t in texts if t)
    except Exception as e:
        return f'ERROR: {e}'

# Transcribe all doctor and patient audio files in the dataframe
doctor_audio_paths = primock_datasets['doctor_audio_path'].tolist()
patient_audio_paths = primock_datasets['patient_audio_path'].tolist()
audio_path = all_datasets_df['audio_file'].tolist()

primock_datasets['Phi-4-ASR-Doctor'] = [transcribe_file_chunked(path, chunk_seconds=100, max_new_tokens=500) for path in tqdm(doctor_audio_paths, desc='Transcribing Doctor')]
primock_datasets['Phi-4-ASR-Patient'] = [transcribe_file_chunked(path, chunk_seconds=100, max_new_tokens=500) for path in tqdm(patient_audio_paths, desc='Transcribing Patient')]
all_datasets_df['Phi-4-ASR'] = [transcribe_file_chunked(path, chunk_seconds=100, max_new_tokens=500) for path in tqdm(audio_path, desc='Transcribing Audio')]

Transcribing Doctor:   0%|          | 0/45 [00:00<?, ?it/s]/home/kelechi/miniconda3/envs/phi4_env/lib/python3.9/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.
Transcribing Patient:   0%|          | 0/45 [00:00<?, ?it/s]/home/kelechi/miniconda3/envs/phi4_env/lib/python3.9/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Transcribing Patient: 100%|██████████| 45/45 [1:37:46<00:00, 130.36s/it]  


## Whisper ASR Model

In [5]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


# use a device id for pipeline (int) and a torch device string for .to()
torch_device = "cuda:0" if torch.cuda.is_available() else "cpu"
device_id = 0 if torch.cuda.is_available() else -1
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
# move the model to the proper device
model.to(torch_device)

processor = AutoProcessor.from_pretrained(model_id)

# create the pipeline; we keep model/tokenizer/feature_extractor explicit
# note: pass device as int (0 for cuda, -1 for cpu) to the pipeline
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device_id,
)



`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


In [ ]:
# Run Whisper ASR on all audio files in the dataframe for both doctor and patient
def run_whisper_asr(audio_paths, desc):
    responses = []
    for audio_path in tqdm(audio_paths, desc=desc):
        try:
            result = pipe(audio_path, return_timestamps=True, chunk_length_s=100)
            responses.append(result.get("text", ""))
        except Exception as e:
            responses.append(f'ERROR: {e}')
    return responses

primock_datasets['Whisper-ASR-Doctor'] = run_whisper_asr(primock_datasets['doctor_audio_path'].tolist(), 'Whisper Doctor Transcribing')
primock_datasets['Whisper-ASR-Patient'] = run_whisper_asr(primock_datasets['patient_audio_path'].tolist(), 'Whisper Patient Transcribing')
all_datasets_df['Whisper-ASR'] = run_whisper_asr(all_datasets_df['audio_file'].tolist(), 'Whisper Audio Transcribing')

Whisper Doctor Transcribing:   0%|          | 0/45 [00:00<?, ?it/s]Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://

In [2]:
# save results to csv
primock_datasets.to_csv('results/phi_4_asr_results_primock.csv', index=False)
all_datasets_df.to_csv('results/phi_4_asr_results_all.csv', index=False)